# SLU15 - Feature Engineering (aka Real World Data): Exercises notebook

## 1 About the data

In this exercise we will be using a synthetic financial dataset used for fraud detection, adapted from [here](https://www.kaggle.com/ntnu-testimon/paysim1).

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('data/transactions.csv')
data.head()

The fields in this dataset have the following meaning:
* **time** - 1 hour of time in the real world. Total 744 hours (30 days simulation).
* **type** - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.
* **risk** - low, medium, high.
* **amount** - amount of the transaction in local currency.
* **availableBalance** - initial balance before the transaction.
* **nameOrig** - customer who started the transaction.
* **nameDest** - customer who is the recipient of the transaction.
* **isFraud** - This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.

In [ ]:
X = data.copy()
y = X.pop('isFraud')

The first thing we want to do is to check the dtypes of our features.

In [ ]:
X.dtypes

## 2 Category dtype in pandas

### Exercise 1: Convert fields into category dtype (graded)

The fields `type` and `risk` are of dtype `object` but can be converted into dtype `category`, as explained in the Learning Notebook. Moreover:
* `type` is a *nominal* categorical field, that is, without any meaningful order;
* `risk` is an *ordinal* categorical field, as its values has a natural order.

In the following exercise, convert both fields into dtype `category` and, in the case of the field `risk`, assign a natural order for its categories.

In [ ]:
def convert_categorical_features(X, nominal_feat='type', ordinal_feat='risk'):

    X_s = X.copy()
    
    ## convert nominal feature to dtype 'category'
    # ...
    ## create list of ordered categories for ordinal feature
    # ordered_cats = ...
    ## convert ordinal feature to dtype 'category'
    # ...
    ## Assign natural order to ordinal feature
    # ...
    
    # YOUR CODE HERE
    raise NotImplementedError()

    return X_s

In [ ]:
"""Check that the solution is correct."""
X_cat_conv = convert_categorical_features(X)

assert X_cat_conv.type.dtype == 'category'
assert X_cat_conv.risk.dtype == 'category'
assert X_cat_conv.risk.min() == 'low'
assert X_cat_conv.risk.max() == 'high'

### Exercise 2: Encode binary target (graded)

The target variable for this dataset is `isFraud`, which was read by pandas as a field of dtype `bool`.

In [ ]:
y.dtype

In this exercise, encode the target variable to be `1` when a transaction is fraudulent and `0` otherwise using the `map` method.

In [ ]:
def encode_target(y):

    y_e = y.copy()
    
    ## create a dictionary mapping the current values to int values
    # enconding_map = ...
    ## change target using the mapping
    # y_e = ...
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return y_e

In [ ]:
"""Check that the solution is correct."""
y_encoded = encode_target(y)

assert y_encoded.dtype == 'int64'
assert y_encoded[120] == 0
assert y_encoded[58128] == 1
assert sum(y_encoded) == 8213

### Exercise 3: Discretize amount field (graded)

The field `amount` is a continuous field, with a distribution which is, not surprisingly, very skewed to the right (remember *skewness* from SLU04?).

In [ ]:
X.amount.plot.hist(bins=100, figsize=(10,6));
plt.xlim(0);
plt.xlabel('amount');
print("The field 'amount' ranges from", X.amount.min(), "to", X.amount.max())

We will deal with the skewness in a bit. Let's first discretize this field in two ways:
* create a new field called `discrete_amount` which is the discretization of the `amount` field, such that the range is between 0 and 99 and the original instances are uniformly distributed;
* create a new field called `binary_amount` which is the binarization of the `amount` field, such that amounts smaller than `10000` become `0` and amounts equal or greater than `10000` become 1.

Use `sklearn` transformers in this exercise.

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer

def discretize_amounts(X):

    X_a = X.copy()
    
    ## create new column `discrete_amount` using suitable transformer
    # discretizer = ...
    # ...
    ## create new column `binary_amount` using suitable transformer
    # binarizer = ...
    # ...
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return X_a

In [ ]:
"""Check that the solution is correct."""
X_am = discretize_amounts(X)

assert X_am.discrete_amount.nunique() == 64
assert X_am.discrete_amount.max() == 99
assert X_am.loc[322179, 'discrete_amount'] == 16
assert X_am.binary_amount.nunique() == 2
assert X_am.binary_amount.max() == 1
assert X_am.loc[322179, 'binary_amount'] == 1

Check the distribution of the two new fields you just calculated:

In [ ]:
X_am.discrete_amount.plot.hist(bins=100, figsize=(10,6));
plt.xlim(0,99);
plt.xlabel('discrete_amount');
plt.title('amount after discretization');

In [ ]:
X_am.binary_amount.plot.hist(figsize=(4,4));
plt.xlim(0,1);
plt.xlabel('binary_amount');
plt.title('amount after binarization');

### Exercise 4: Scale amount field (graded)

In the Learning Notebook, you also learned that numerical data can be scaled. 

In this exercise, let's scale the field `amount` in three different ways and compare the results:
* create a new field called `minmaxscaled_amount` which scales uniformly the `amount` field such that the values range from 0 to 1;
* create a new field called `standardscaled_amount` which scales the `amount` field such that the *mean* is 0 and the standard deviation is 1;
* create a new field called `robustscaled_amount` which scales the `amount` field such that the *median* is 0 and it is scaled according to the Interquartile Range.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

def scale_amounts(X):

    X_s = X.copy()
    
    ## create new column `minmaxscaled_amount` using suitable transformer
    # ...
    ## create new column `standardscaled_amount` using suitable transformer
    # ...
    ## create new column `robustscaled_amount` using suitable transformer
    # ...
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return X_s

In [ ]:
"""Check that the solution is correct."""
X_scaled = scale_amounts(X)

assert X_scaled.minmaxscaled_amount.min() == 0
assert X_scaled.minmaxscaled_amount.max() == 1
assert math.isclose(X_scaled.minmaxscaled_amount.mean(), 0.0032, abs_tol = 0.0001)
assert math.isclose(X_scaled.loc[12345, 'minmaxscaled_amount'], 0.00012, abs_tol = 0.00001)
assert math.isclose(X_scaled.standardscaled_amount.min(), -0.298, abs_tol = 0.001)
assert math.isclose(X_scaled.standardscaled_amount.max(), 92.75, abs_tol = 0.01)
assert math.isclose(X_scaled.standardscaled_amount.mean(), 0, abs_tol = 0.001)
assert math.isclose(X_scaled.loc[12345, 'standardscaled_amount'], -0.287, abs_tol = 0.01)
assert math.isclose(X_scaled.robustscaled_amount.min(), -0.386, abs_tol = 0.001)
assert math.isclose(X_scaled.robustscaled_amount.max(), 305.86, abs_tol = 0.01)
assert math.isclose(X_scaled.robustscaled_amount.mean(), 0.595, abs_tol = 0.002)
assert math.isclose(X_scaled.loc[12345, 'robustscaled_amount'], -0.349, abs_tol = 0.01)

Plot the distributions for the new fields you just calculated:

In [ ]:
X_scaled.minmaxscaled_amount.plot.hist(bins=50, figsize=(10,6));
plt.xlim(0,1);
plt.xlabel('minmaxscaled_amount');
plt.title('amount after min-max scaling');

In [ ]:
X_scaled.standardscaled_amount.plot.hist(bins=50, figsize=(10,6));
plt.xlabel('standardscaled_amount');
plt.title('amount after standard scaling');

In [ ]:
X_scaled.robustscaled_amount.plot.hist(bins=50, figsize=(10,6));
plt.xlabel('robustscaled_amount');
plt.title('amount after robust scaling');

### Exercise 5: Ordinal encode risk feature

Finally, let's deal with the categorical features.

First, create a new field called `risk_encoded` which is the result of ordinal encoding of the `risk` feature.

In [ ]:
import category_encoders as ce

def encode_risk(X):

    X_r = X.copy()
    
    # create new column using suitable transformer
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return X_r

In [ ]:
"""Check that the solution is correct."""
X_risk = encode_risk(X)

assert X_risk.risk_encoded.dtype == int
assert X_risk.risk_encoded.min() == 1
assert X_risk.risk_encoded.max() == 3
assert X_risk.loc[12345, 'risk_encoded'] == 3

### Exercise 6: One-hot encode type feature

Finally, perform a one-hot encoding of the `type` feature. Pay attention to the following points:
* return the original DataFrame `X`, but with the `type` feature replaced by the new ones resulting from the one-hot encoding;
* make sure the new features have names of the form `type_<value>`, where `<value>` is the category being indicated by that feature.

In [ ]:
def encode_type(X):

    X_t = X.copy()
    
    # perform one-hot encoding in X_t
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return X_t

In [ ]:
"""Check that the solution is correct."""
X_type = encode_type(X)

assert X_type.shape[1] > 10
assert X_type.type_CASH_IN.sum() == 140058
assert X_type['type_-1'].sum() == 0
assert X_type.loc[12345, 'type_TRANSFER'] == 0
assert X_type.loc[362312, 'type_CASH_OUT'] == 1